# Data

In [1]:
# Import of libraries
from __future__ import print_function 
from ortools.sat.python import cp_model
from ortools.linear_solver import pywraplp
import time
import numpy as np
import pandas as pd
import xarray as xr
start = time.time()

In [2]:
# Data Import from Excel

archivo_input = 'Asignacion prof 1 curso.xlsx'
df_pref = pd.read_excel(archivo_input, 'PREFERENCIAS', index_col=(0,1,2))
df_pref = df_pref.drop('INICIO',axis=1)
df_pref = df_pref.drop('FIN',axis=1)
df_pref

df_pref = df_pref.rename_axis('DIA',axis=1)
df_pref = df_pref.stack()
xa_pref = df_pref.to_xarray()
print("Dimensions of xa_pref: ", xa_pref.dims)
print("Shape of xa_pref: ", xa_pref.shape)
print("")

ncursos = xa_pref.shape[0] # posicion i
nprof = xa_pref.shape[1] # posicion j
nturnos = xa_pref.shape[2] # posicion k
ndias = xa_pref.shape[3] # posicion l

df_cargahs = pd.read_excel(archivo_input, 'CARGA_HS', index_col=(0,1))
df_cargahs = df_cargahs.drop('MATERIA',axis=1)
xa_cargahs = df_cargahs.to_xarray()
xa_cargahs = xa_cargahs['HORAS'].fillna(0)
print("Dimensions of xa_cargahs: ", xa_cargahs.dims)
print("Shape of xa_cargahs: ", xa_cargahs.shape)

Dimensions of xa_pref:  ('CURSO', 'PROF', 'TURNO', 'DIA')
Shape of xa_pref:  (2, 11, 5, 5)

Dimensions of xa_cargahs:  ('CURSO', 'PROF')
Shape of xa_cargahs:  (2, 11)


# Model

In [3]:
# MODEL CREATION

model = cp_model.CpModel()
#model = pywraplp.Solver('modelo_asigna', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
#pywraplp.Solver.GLOP_LINEAR_PROGRAMMING
#pywraplp.Solver.CLP_LINEAR_PROGRAMMING
#pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING

In [4]:
ndias

5

In [5]:
%%time
# VARIABLES

x = []
for i in range(ncursos):
    a=[]
    for j in range(nprof):
        b = []
        for k in range(nturnos):
            c = []
            for l in range(ndias):
                c.append(model.NewBoolVar("x[%i,%i,%i,%i]" % (i, j, k, l)))
            b.append(c)
        a.append(b)
    x.append(a)
    
cambios = []
for i in range(ncursos):
    a=[]
    for j in range(nprof):
        b = []
        for k in range(round(nturnos/2)):
            c = []
            for l in range(ndias):
                c.append(model.NewBoolVar("cambios[%i,%i,%i,%i]" % (i, j, k, l)))
            b.append(c)
        a.append(b)
    cambios.append(a)
    
print("Shape de x: ", np.array(x).shape)
print("Shape de cambios: ", np.array(cambios).shape)
#print('Number of variables =', model.NumVariables())

Shape de x:  (2, 11, 5, 5)
Shape de cambios:  (2, 11, 2, 5)
Wall time: 11 ms


In [6]:
%%time
# LINKEAGE OF VARIABLES

[model.Add(x[i][j][0][l] - x[i][j][1][l] == 0).OnlyEnforceIf(cambios[i][j][0][l].Not())
 for i in range(ncursos) for j in range(nprof) for l in range(ndias)]

[model.Add(x[i][j][2][l] - x[i][j][3][l] == 0).OnlyEnforceIf(cambios[i][j][1][l].Not())
 for i in range(ncursos) for j in range(nprof) for l in range(ndias)]

print("Cambios linked")

Cambios linked
Wall time: 14 ms


In [7]:
%%time
# CONSTRAINTS

pref = np.array(xa_pref)
cargahs = np.array(xa_cargahs)

# Horas acumuladas = Carga horaria de prof/materia x curso
[model.Add(sum(x[i][j][k][l] for k in range(nturnos) for l in range(ndias)) == cargahs[i,j]) for i in range(ncursos) for j in range(nprof)]

# Horas asignadas <= Horas preferidas x prof
[model.Add(x[i][j][k][l] <= pref[i,j,k,l]) for i in range(ncursos) for j in range(nprof) for k in range(nturnos) for l in range(ndias)]

# Solo un profesor por turno
[model.Add(sum(x[i][j][k][l] for j in range(nprof)) <= 1) for i in range(ncursos) for k in range(nturnos) for l in range(ndias)]

# Un profesor solo puede estar en un curso
[model.Add(sum(x[i][j][k][l] for i in range(ncursos)) <= 1) for j in range(nprof) for k in range(nturnos) for l in range(ndias)]

# Un prof/materia no puede tener todas sus horas en 2 dias consecutivos
#for m in range(ndias):
[model.Add(sum(x[i][j][k][l] for k in range(nturnos) for l in range(0,2)) <= (max(2,cargahs[i,j])-1)) for i in range(ncursos) for j in range(nprof)]
[model.Add(sum(x[i][j][k][l] for k in range(nturnos) for l in range(1,3)) <= (max(2,cargahs[i,j])-1)) for i in range(ncursos) for j in range(nprof)]
[model.Add(sum(x[i][j][k][l] for k in range(nturnos) for l in range(2,4)) <= (max(2,cargahs[i,j])-1)) for i in range(ncursos) for j in range(nprof)]
[model.Add(sum(x[i][j][k][l] for k in range(nturnos) for l in range(3,5)) <= (max(2,cargahs[i,j])-1)) for i in range(ncursos) for j in range(nprof)]

#[model.AddImplication(x[i][j][0][l], x[i][j][1][l]) for i in range(ncursos) for j in range(nprof) for l in range(ndias)]
#[model.AddImplication(x[i][j][2][l], x[i][j][3][l]) for i in range(ncursos) for j in range(nprof) for l in range(ndias)]


print('Number of constraints =')

Number of constraints =
Wall time: 59.8 ms


In [ ]:
%%time
# OBJECTIVE FUNCTION

#model.SetTimeLimit(100000)
#(np.array(x) * xa_pref.values).sum() + 
model.Maximize((np.array(x) * xa_pref.values).sum() - np.array(cambios).sum()*2)
solver = cp_model.CpSolver()
status = solver.Solve(model)
print('Solve status: %s' % solver.StatusName(status))
if status == cp_model.OPTIMAL:
    print('Optimal objective value: %i' % solver.ObjectiveValue())

In [ ]:
# OUTPUT

output=[]

for i in range(ncursos):
    for j in range(nprof):
        for k in range(nturnos):
            for l in range(ndias):
                output.append(solver.Value(x[i][j][k][l]))
                #output.append(x[i][j][k][l].solution_value())
            
m_out = np.array(output).reshape(ncursos, nprof, nturnos, ndias)

print("Shape of m_out: ", m_out.shape)

In [ ]:
# EXPORT TO EXCEL

xa_out = xr.DataArray(m_out, dims=xa_pref.dims, coords=xa_pref.coords) + df_pref.to_xarray()*0
xa_out_st = xa_out.stack(id=(xa_out.dims[0],xa_out.dims[1],xa_out.dims[2]))
df = xa_out_st.T.to_pandas()
#df = df.dropna(0,how='all')
df = df.reset_index()
writer = pd.ExcelWriter('Horarios_asignados.xlsx')
df.to_excel(writer,'Sheet1')
writer.save()
print("Shape of export to xls: ", xa_out_st.T.shape)

In [ ]:
# TIEMPO DE CORRIDA

end = time.time()
print("Time = ", round(end - start, 2), "seconds")

In [ ]:
print("Turnos seguidos 1&2: ", np.array(np.array(xa_out)[:,:,0:2,:].sum(axis=2) == 2).sum())
print("Turnos seguidos 3&4: ", np.array(np.array(xa_out)[:,:,2:4,:].sum(axis=2) == 2).sum())

#print("Turnos seguidos 1&2: ", np.count_nonzero(np.array(xa_out)[:,:,0:2,:].sum(axis=2) == 2))
#print("Turnos seguidos 3&4: ", np.count_nonzero(np.array(xa_out)[:,:,2:4,:].sum(axis=2) == 2))

#print("Turnos seguidos 1&2: ", (np.array(xa_out)[:,:,0:1,:] * np.array(xa_out)[:,:,1:2,:]).sum())
#print("Turnos seguidos 1&2: ", (np.array(xa_out)[:,:,2:3,:] * np.array(xa_out)[:,:,3:4,:]).sum())